# ***Extra Credit Assignment 1***

In [ ]:
import os
print(os.getcwd())  # Check current working directory
os.chdir('/Users/devynmiller/Downloads/ec1-cpsc542')  # Change to your project directory

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import importlib


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# **Preprocessing**

### Load data

In [ ]:
#preprocessing.py
import src.dataloader_eda as dataloader_eda
importlib.reload(dataloader_eda)

test_path, train_path, train = dataloader_eda.load_data()

### Augmentation and EDA

In [ ]:
# Test the generator
example, label = next(dataloader_eda.data_generator(batch_size=1))
img = example['image'][0]
bbox_coords = label['coords'][0]

In [ ]:
dataloader_eda.display_image_from_file("vid_4_10520.jpg")

dataloader_eda.display_image(img, bbox_coords=bbox_coords, norm=True)
dataloader_eda.display_grid(train)

# **Model**

### Model

In [ ]:
import model as model
importlib.reload(model)

model, test = model.modeling()

### Prediction

In [ ]:
import src.predict as predict
importlib.reload(predict)

image_path = "example_image.jpg"
predicted_bbox = predict.predict_bounding_box(image_path)
print(f"Predicted bounding box for {image_path}: {predicted_bbox}")

### Training

In [ ]:
import src.train as train_model
importlib.reload(train_model)
import json

train.train_model()

# **Validation**

### Loss Metrics/Graphs

In [ ]:
import src.evaluation as evaluate
importlib.reload(evaluate)

# Loop through each of the 5 best models
for i in range(5):
    model_path = f'best_model_{i}.h5'
    history, best_3, worst_3, display_with_boxes, generate_gradcam = evaluate.evaluate_model(model_path)
    
    # Plot training and validation accuracy and loss
    evaluate.plot_model_metrics(history, f'Best Model {i}')
    
    # Display best 3 bounding boxes
    print(f"Best 3 for model {i}")
    for iou, image, pred_box, true_box in best_3:
        display_with_boxes(image, pred_box, true_box)

    # Display worst 3 bounding boxes
    print(f"Worst 3 for model {i}")
    for iou, image, pred_box, true_box in worst_3:
        display_with_boxes(image, pred_box, true_box)
    
    # Generate and display GradCAM visualization

    # Inside the loop for each model
    img = load_img('path_to_your_image.jpg', target_size=(224, 224))  # Adjust target_size as per your model's input
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Model expects a batch of images, so add batch dimension

    layer_name = 'conv2d_9'  # Replace with your layer name
    gradcam_image = generate_gradcam(model_path, img_array, layer_name)

    # Display GradCAM image
    plt.imshow(gradcam_image)
    plt.title(f'GradCAM for Best Model {i}')
    plt.show()